In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
functionals = ["blyp",
               "PBE",
               "LC-BLYP",
               "LC-wPBE",
               "bmk",
               "m062x",
               "m11",
               "b3lyp",
               "PBE0",
               "cam-b3lyp",
               "wB97XD",
                ]

func_to_name = {
    'blyp': "BLYP",
    'bmk': "BMK",
    'm11': "M11",
    'm062x': "M062X",
    'b3lyp': "B3LYP",
    'PBE0': 'PBE0',
    'cam-b3lyp': 'CAM-B3LYP',
    'LC-wPBE': 'LC-wPBE',
    'LC-BLYP':'LC-BLYP',
    'wB97XD': 'wB97XD',
    'PBE': 'PBE'
}

In [ ]:
dfS1 = pd.read_csv("../data/S1.csv")
dfT1 = pd.read_csv("../data/T1.csv")
dfT2 = pd.read_csv("../data/T2.csv")

dfFC = pd.DataFrame()
dfSC = pd.DataFrame()

dfFC["DRC"] = dfS1["DRC"]
dfFC["S2"] = dfS1["S2"]

dfSC["DRC"] = dfS1["DRC"]
dfSC["S2"] = dfS1["S2"]


for f in functionals:
    dfFC['FC({})'.format(f)] = 2*dfT1["T1({})".format(f)] - dfS1["S1({})".format(f)]
    dfFC['FC({}, TDA)'.format(f)] = 2*dfT1["T1({}, TDA)".format(f)] - dfS1["S1({}, TDA)".format(f)]
    
    dfSC['SC({})'.format(f)] = 2*dfT1["T1({})".format(f)] - dfT2["T2({})".format(f)]
    dfSC['SC({}, TDA)'.format(f)] = 2*dfT1["T1({}, TDA)".format(f)] - dfT2["T2({}, TDA)".format(f)] 
    
dfFC["FC(MC)"] = 2*dfT1["T1(MC)"] - dfS1["S1(MC)"]
dfSC['SC(MC)'.format(f)] = 2*dfT1["T1(MC)"] - dfT2["T2(MC)"]

In [ ]:
def calc_mae_mse(df,state, tda = False):
    if (tda): 
        print("{}, TDA".format(state))
    else:
        print(state)
    out_df = pd.DataFrame()
    func, mae, mse = [], [], []
    for f in functionals:
        func.append(func_to_name[f])
        lab = '{}({})'.format(state, f)
        if (tda == True):
            lab = '{}({}, TDA)'.format(state, f)
        deltaE = df["{}(MC)".format(state)] - df[lab]
        mse.append(deltaE.mean())
        mae.append(np.abs(deltaE.mean()))
    
    out_df["Functional"] = func
    out_df["MAE, eV"] = mae
    out_df["MSE, eV"] = mse
    return out_df
        
            

In [ ]:
calc_mae_mse(dfS1,"S1")

In [ ]:
calc_mae_mse(dfS1,"S1", tda=True)

In [ ]:
calc_mae_mse(dfT1,"T1")

In [ ]:
calc_mae_mse(dfT1,"T1", tda=True)

In [ ]:
calc_mae_mse(dfT2,"T2")

In [ ]:
calc_mae_mse(dfT2,"T2", tda=True)

In [ ]:
calc_mae_mse(dfFC,"FC")

In [ ]:
calc_mae_mse(dfFC,"FC", tda=True)

In [ ]:
calc_mae_mse(dfSC,"SC")

In [ ]:
calc_mae_mse(dfSC,"SC",tda=True)

In [ ]:
s1mse = calc_mae_mse(dfS1,"S1")
t1mse = calc_mae_mse(dfT1,"T1")
t2mse = calc_mae_mse(dfT2,"T2")

s1msetda = calc_mae_mse(dfS1,"S1", tda=True)
t1msetda = calc_mae_mse(dfT1,"T1", tda=True)
t2msetda = calc_mae_mse(dfT2,"T2", tda=True)

fig = plt.figure(figsize=(20,10))
fig.tight_layout(pad=5.0)
X = np.arange(s1mse.shape[0])
ax = fig.add_axes([0,0,1,1])

c1 = 'c'
c2 = 'm'
c3 = 'orange'

ax.tick_params(axis='both', labelsize=22)

ax.bar(X - 0.25, s1mse["MAE, eV"], color = c1, width = 0.25, label = "S1", edgecolor='black')
ax.bar(X + 0.00, t1mse["MAE, eV"], color = c2, width = 0.25, label = "T1", edgecolor='black')
ax.bar(X + 0.25, t1mse["MAE, eV"], color = c3, width = 0.25, label = "T2", edgecolor='black')

ax.bar(X - 0.25, s1msetda["MAE, eV"], color = 'none', width = 0.25, label = "TDA", hatch='o', edgecolor='black')
ax.bar(X + 0.00, t1msetda["MAE, eV"], color = 'none', width = 0.25, hatch='o', edgecolor='black')
ax.bar(X + 0.25, t1msetda["MAE, eV"], color = 'none', width = 0.25, hatch='o', edgecolor='black')

#ax.bar(X - 0.25, s1msetda["MAE, eV"], width = 0.25, label = "S1", hatch='x', edgecolor='black')

ax.set_xticks(X)
ax.set_xticklabels(list(s1mse["Functional"]), rotation=45, fontweight='bold')

ax.set_ylabel("MAE, eV", fontsize=24)
ax.axhline(0.2, -100, 100, linestyle='--',linewidth=3, color = 'red')
ax.set_ylim(0,1.5)
l = ax.legend(fontsize=24)
plt.savefig("MAE.png")
plt.show()

In [ ]:
fig = plt.figure(figsize=(20,15))
X = np.arange(s1mse.shape[0])
ax = fig.add_axes([0,0,1,1])

c1 = 'c'
c2 = 'm'
c3 = 'orange'

ax.tick_params(axis='both', labelsize=22)

ax.bar(X - 0.25, s1mse["MSE, eV"], color = c1, width = 0.25, label = "S1", edgecolor='black')
ax.bar(X + 0.00, t1mse["MSE, eV"], color = c2, width = 0.25, label = "T1", edgecolor='black')
ax.bar(X + 0.25, t1mse["MSE, eV"], color = c3, width = 0.25, label = "T2", edgecolor='black')

ax.bar(X - 0.25, s1msetda["MSE, eV"], color = 'none', width = 0.25, label = "TDA", hatch='o', edgecolor='black')
ax.bar(X + 0.00, t1msetda["MSE, eV"], color = 'none', width = 0.25, hatch='o', edgecolor='black')
ax.bar(X + 0.25, t1msetda["MSE, eV"], color = 'none', width = 0.25, hatch='o', edgecolor='black')

#ax.bar(X - 0.25, s1msetda["MAE, eV"], width = 0.25, label = "S1", hatch='x', edgecolor='black')

ax.set_xticks(X)
ax.set_xticklabels(list(s1mse["Functional"]), rotation=45, fontweight='bold')

ax.axhline(0.2, -100, 100, linestyle='--',linewidth=3, color = 'red')
ax.axhline(-0.2, -100, 100, linestyle='--',linewidth=3, color = 'red')

ax.axhline(0.0, -100, 100, linewidth= 0.5, color = 'black')
ax.set_ylim(-1.5,1.5)

ax.set_ylabel("MSE, eV", fontsize=24)
l = ax.legend(fontsize=24)
plt.savefig("MSE.png")
plt.show()

In [ ]:
FCmse = calc_mae_mse(dfFC,"FC")
SCmse = calc_mae_mse(dfSC,"SC")

FCmsetda = calc_mae_mse(dfFC,"FC", tda=True)
SCmsetda = calc_mae_mse(dfSC,"SC", tda=True)

fig = plt.figure(figsize=(20,10))
fig.tight_layout(pad=5.0)
X = np.arange(s1mse.shape[0])
ax = fig.add_axes([0,0,1,1])

c1 = 'c'
c2 = 'm'

ax.tick_params(axis='both', labelsize=22)

ax.bar(X - 0.25/2, FCmse["MAE, eV"], color = c1, width = 0.25, label = "1$^{st}$ feasibility condition", 
       edgecolor='black')
ax.bar(X + 0.25/2, SCmse["MAE, eV"], color = c2, width = 0.25, label = "2$^{nd}$ feasibility condition", 
       edgecolor='black')

ax.bar(X - 0.25/2, FCmsetda["MAE, eV"], color = 'none', width = 0.25, label = "TDA", hatch='o', edgecolor='black')
ax.bar(X + 0.25/2, SCmsetda["MAE, eV"], color = 'none', width = 0.25, hatch='o', edgecolor='black')
#ax.bar(X + 0.25, t1msetda["MAE, eV"], color = 'none', width = 0.25, hatch='o', edgecolor='black')

#ax.bar(X - 0.25, s1msetda["MAE, eV"], width = 0.25, label = "S1", hatch='x', edgecolor='black')

ax.set_xticks(X)
ax.set_xticklabels(list(s1mse["Functional"]), rotation=45, fontweight='bold')

ax.set_ylabel("MAE, eV", fontsize=24)
ax.axhline(0.2, -100, 100, linestyle='--',linewidth=3, color = 'red')
ax.set_ylim(0,3.2)
l = ax.legend(fontsize=24)
#plt.savefig("MAE.png")
plt.show()

In [ ]:
FCmse = calc_mae_mse(dfFC,"FC")
SCmse = calc_mae_mse(dfSC,"SC")

FCmsetda = calc_mae_mse(dfFC,"FC", tda=True)
SCmsetda = calc_mae_mse(dfSC,"SC", tda=True)

fig = plt.figure(figsize=(20,10))
fig.tight_layout(pad=5.0)
X = np.arange(s1mse.shape[0])
ax = fig.add_axes([0,0,1,1])

c1 = 'c'
c2 = 'm'

ax.tick_params(axis='both', labelsize=22)

ax.bar(X - 0.25/2, FCmse["MSE, eV"], color = c1, width = 0.25, label = "1$^{st}$ feasibility condition", 
       edgecolor='black')
ax.bar(X + 0.25/2, SCmse["MSE, eV"], color = c2, width = 0.25, label = "2$^{nd}$ feasibility condition", 
       edgecolor='black')

ax.bar(X - 0.25/2, FCmsetda["MSE, eV"], color = 'none', width = 0.25, label = "TDA", hatch='o', edgecolor='black')
ax.bar(X + 0.25/2, SCmsetda["MSE, eV"], color = 'none', width = 0.25, hatch='o', edgecolor='black')

ax.set_xticks(X)
ax.set_xticklabels(list(s1mse["Functional"]), rotation=45, fontweight='bold')

ax.set_ylabel("MSE, eV", fontsize=24)
ax.axhline(0.2, -100, 100, linestyle='--',linewidth=3, color = 'red')
ax.axhline(-0.2, -100, 100, linestyle='--',linewidth=3, color = 'red')
ax.axhline(0.0, -100, 100, linewidth= 0.5, color = 'black')
ax.set_ylim(-.5,3.2)
l = ax.legend(fontsize=24)
plt.savefig("MAE_FC.png")
plt.show()